In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
import pandas_gbq
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1855/4151465537.py:10: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


In [3]:
df_bq=df_bq2[(df_bq2["GP"]=='Belgian Grand Prix')&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1855/2085494510.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GP["LapNumber"]=GP["LapNumber"].astype(int)


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
1209,0 days 00:34:16.282000,ALB,23,0 days 00:01:27.279000,1,1.0,NaT,NaT,NaT,0 days 00:00:31.947000,0 days 00:00:33.998000,NaT,0 days 00:33:42.286000,0 days 00:34:16.468000,311.0,219.0,162.0,296.0,False,SOFT,1.0,True,Red Bull Racing,0 days 00:32:48.804000,NaT,14,5.0,False,,False,False,2020,Styrian Grand Prix
44,0 days 00:36:16.996000,ALB,23,0 days 00:02:00.714000,2,1.0,NaT,NaT,0 days 00:00:24.352000,0 days 00:01:01.020000,0 days 00:00:35.342000,0 days 00:34:40.634000,0 days 00:35:41.654000,0 days 00:36:16.996000,195.0,135.0,190.0,125.0,True,SOFT,2.0,True,Red Bull Racing,0 days 00:34:16.282000,NaT,4,5.0,False,,False,False,2020,Styrian Grand Prix
45,0 days 00:38:02.615000,ALB,23,0 days 00:01:45.619000,3,1.0,NaT,NaT,0 days 00:00:26.822000,0 days 00:00:46.446000,0 days 00:00:32.351000,0 days 00:36:43.818000,0 days 00:37:30.264000,0 days 00:38:02.615000,124.0,144.0,271.0,242.0,True,SOFT,3.0,True,Red Bull Racing,0 days 00:36:16.996000,NaT,41,5.0,False,,False,False,2020,Styrian Grand Prix
46,0 days 00:39:13.623000,ALB,23,0 days 00:01:11.008000,4,1.0,NaT,NaT,0 days 00:00:17.768000,0 days 00:00:31.514000,0 days 00:00:21.726000,0 days 00:38:20.383000,0 days 00:38:51.897000,0 days 00:39:13.623000,304.0,230.0,273.0,293.0,True,SOFT,4.0,True,Red Bull Racing,0 days 00:38:02.615000,NaT,12,5.0,False,,False,True,2020,Styrian Grand Prix
47,0 days 00:40:24.368000,ALB,23,0 days 00:01:10.745000,5,1.0,NaT,NaT,0 days 00:00:17.604000,0 days 00:00:31.382000,0 days 00:00:21.759000,0 days 00:39:31.227000,0 days 00:40:02.609000,0 days 00:40:24.368000,291.0,232.0,273.0,286.0,True,SOFT,5.0,True,Red Bull Racing,0 days 00:39:13.623000,NaT,1,5.0,False,,False,True,2020,Styrian Grand Prix


In [4]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [5]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [6]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [7]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
1209,ALB,1,0.0,31.947,33.998,00:01:27.279000,87.279,SOFT,2020,Styrian Grand Prix,5.0,0.0,5.0,5.0
1207,BOT,1,0.0,31.618,32.731,00:01:25.391000,85.391,SOFT,2020,Styrian Grand Prix,4.0,0.0,4.0,4.0
1220,GAS,1,0.0,32.537,37.103,00:01:32.014000,92.014,SOFT,2020,Styrian Grand Prix,8.0,0.0,8.0,9.0
1219,GIO,1,0.0,34.984,42.088,00:01:40.400000,100.400,MEDIUM,2020,Styrian Grand Prix,15.0,0.0,15.0,16.0
1218,GRO,1,0.0,32.662,43.243,00:01:43.860000,103.860,SOFT,2020,Styrian Grand Prix,19.0,0.0,9.0,19.0


In [14]:
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [15]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,Styrian Grand Prix,2020,2,00:01:58.343000,118.343,NOR,SOFT,23.363,GAS,SOFT,44.034,LEC,HARD,34.742,BOT,SOFT
1,Styrian Grand Prix,2020,3,00:01:33.413000,93.413,RAI,MEDIUM,26.068,GRO,SOFT,41.319,LEC,HARD,25.447,RAI,MEDIUM
2,Styrian Grand Prix,2020,4,00:01:08.631000,68.631,HAM,SOFT,17.314,HAM,SOFT,30.323,HAM,SOFT,20.994,HAM,SOFT
3,Styrian Grand Prix,2020,5,00:01:08.968000,68.968,HAM,SOFT,17.367,HAM,SOFT,30.570,VER,SOFT,20.934,HAM,SOFT
4,Styrian Grand Prix,2020,6,00:01:08.897000,68.897,HAM,SOFT,17.269,BOT,SOFT,30.458,VER,SOFT,20.981,HAM,SOFT


In [16]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,Styrian Grand Prix,2020,2,00:01:58.343000,118.343,NOR,SOFT,23.363,GAS,SOFT,44.034,LEC,HARD,34.742,BOT,SOFT,#FF8700
1,Styrian Grand Prix,2020,3,00:01:33.413000,93.413,RAI,MEDIUM,26.068,GRO,SOFT,41.319,LEC,HARD,25.447,RAI,MEDIUM,#F596C8
2,Styrian Grand Prix,2020,4,00:01:08.631000,68.631,HAM,SOFT,17.314,HAM,SOFT,30.323,HAM,SOFT,20.994,HAM,SOFT,#00D2BE
3,Styrian Grand Prix,2020,5,00:01:08.968000,68.968,HAM,SOFT,17.367,HAM,SOFT,30.570,VER,SOFT,20.934,HAM,SOFT,#00D2BE
4,Styrian Grand Prix,2020,6,00:01:08.897000,68.897,HAM,SOFT,17.269,BOT,SOFT,30.458,VER,SOFT,20.981,HAM,SOFT,#00D2BE


In [17]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
66,Styrian Grand Prix,2020,68,00:01:05.619000,65.619,SAI,SOFT,16.784,PER,MEDIUM,28.917,SAI,SOFT,19.720,SAI,SOFT,#FF8700
68,Styrian Grand Prix,2020,70,00:01:06.145000,66.145,VER,SOFT,16.768,NOR,MEDIUM,29.200,VER,SOFT,20.025,VER,SOFT,#1E41FF
69,Styrian Grand Prix,2020,71,00:01:06.599000,66.599,VER,SOFT,16.688,RIC,SOFT,29.195,VER,SOFT,20.311,VER,SOFT,#1E41FF
64,Styrian Grand Prix,2020,66,00:01:07.018000,67.018,HAM,MEDIUM,16.745,BOT,MEDIUM,29.815,HAM,MEDIUM,20.163,ALB,MEDIUM,#00D2BE
63,Styrian Grand Prix,2020,65,00:01:07.286000,67.286,PER,MEDIUM,16.847,STR,MEDIUM,29.862,NOR,MEDIUM,20.225,ALB,MEDIUM,#F596C8


In [18]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=11)
    ))
 )

# fig2.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": LapTime_ordered.T.values
#                                 if c == "All"
#                                 else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )

fig2.update_layout(
    title="Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="Black"
    ),
    height=780,
    width=1200,
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)


fig2.show()

In [19]:
fig2.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi GP Plot

In [16]:
#2024
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [17]:
#2024
race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2024
for i in race_multi:

    df_bq=df_bq2[(df_bq2["GP"]==i)&(df_bq2["Year"]==year_multi)]
    GP=df_bq
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
    GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
    GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
    GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
    GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
    GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

    GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
    GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
    GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()
    GP_Rk=GP[GP["LapTime"]!="NaT"]
    GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
    GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
    GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)

    GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
    GP_Sectors=GP_Sectors.fillna(0)
    GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()
    Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
    Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
    Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
    Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
    Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
    Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
    Fast_Laps=Fast_Laps.reset_index(drop=True)
    Fast_Laps.head()
    Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
    Fast_Laps.head()
    LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
    LapTime_ordered.head()
    import plotly.graph_objects as go

    fig2 = go.Figure(go.Table
    (header={"values": LapTime_ordered.columns},
    cells=dict(
        values=LapTime_ordered.T.values,
        fill_color=[LapTime_ordered.color],
        align='center', font=dict(color='white', size=11)
        ))
    )
    fig2.update_layout(
        title="Fastest Lap and Sector Times Ordered By Lap Times For The {} {}".format(year_multi, i),
            title_x=0.5,
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=13,
            color="Black"
        ),
        height=780,
        width=1200,
        title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5),
    )
    fig2.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6958/3605435136.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6958/3605435136.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6958/3605435136.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [14]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps.columns},
#   cells=dict(
#     values=Fast_Laps.T.values,
#     fill_color=[Fast_Laps.color],
#     align='center', font=dict(color='white', size=11)
#     ))
#  )

# # fig.update_layout(
# #     updatemenus=[
# #         {
# #             "buttons": [
# #                 {
# #                     "label": c,
# #                     "method": "update",
# #                     "args": [
# #                         {
# #                             "cells": {
# #                                 "values": Fast_Laps.T.values
# #                                 if c == "All"
# #                                 else Fast_Laps.loc[Fast_Laps["LapNumber"].eq(c)].T.values
# #                             }
# #                         }
# #                     ],
# #                 }
# #                 for c in ["All"] + Fast_Laps["LapNumber"].unique().tolist()
# #             ]
# #         }
# #     ]
# # )
# fig.update_layout(
#     title="Fastest Lap and Sector Times For Ordered By Lap Number",
#         title_x=0.5,
#         # template="plotly_dark",

#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=13,
#         color="Black"
#     ),
#     height=780,
#     width=1200,
#     title_font_family="PT Sans Narrow",
#     margin=dict(l=5, r=5, t=35, b=5),
# )

# fig.show()

In [162]:
# fig.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Number.html".format(year,gp))

In [163]:
# #get a shortened view for fastest drivers in the entire lap, S1, S2, and S3

# Fast_Laps_short=Fast_Laps[["GP","Year","LapNumber","Fastest Lap","Fastest S1","Fastest S2","Fastest S3","color"]].sort_values(by=["LapNumber"])
# Fast_Laps_short=Fast_Laps_short.reset_index(drop=True)
# Fast_Laps_short.head()

In [164]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps_short.columns},
#   cells=dict(
#     values=Fast_Laps_short.T.values,
#     height=50,
#     fill_color=[Fast_Laps_short.color],
#     align='center', font=dict(color='white', size=20)
#     ))
#  )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": Fast_Laps_short.T.values
#                                 if c == "All"
#                                 else Fast_Laps_short.loc[Fast_Laps_short["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + Fast_Laps_short["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )
# fig.update_layout(
#     title="Fastest Lap and Sector Times",
#         title_x=0.5,
#     yaxis = dict(tickfont = dict(size=20)),
#     xaxis = dict(tickfont = dict(size=20)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=20,
#         color="Black"
#     ),
#     height=2500,
#     title_font_family="PT Sans Narrow"
# )

# fig.show()

In [107]:
# #get a view for fastest lap in the entire race
# Fastest_Lap_Race=GP_Rk[GP_Rk["Race_RK"]==1][["Year","GP","Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound"]].reset_index(drop=True)
# Fastest_Lap_Race

In [108]:
# #Multiple plotly filters code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "y": 1 - (i / 5),
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "restyle",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df[menu].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df[menu].unique().tolist()
#             ],
#         }
#         for i, menu in enumerate(["Channel_type", "Date"])
#     ]
# )

In [109]:
# #Single Plotly filter code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df["Channel_type"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df["Channel_type"].unique().tolist()
#             ]
#         }
#     ]
# )